In [1]:
import json
import pandas as pd
import numpy as np
import random
import math
#from google.colab import drive
#drive.mount('/content/gdrive')
#path ="gdrive/My Drive/Colab Notebooks/network_science/"

In [2]:
#reading the main data (covid19india) in form of JSON
with open('raw_data2.json') as f:
    data = json.load(f)


In [3]:
#preview the json raw data
data ['raw_data']

[{'agebracket': '48',
  'backupnotes': '',
  'contractedfromwhichpatientsuspected': '',
  'currentstatus': 'Hospitalized',
  'dateannounced': '20/04/2020',
  'detectedcity': '',
  'detecteddistrict': 'Bhadrak',
  'detectedstate': 'Odisha',
  'estimatedonsetdate': '',
  'gender': 'M',
  'nationality': '',
  'notes': 'Details awaited',
  'numcases': '1',
  'patientnumber': '17307',
  'source1': 'https://twitter.com/the_hindu/status/1252059855503806472',
  'source2': '',
  'source3': '',
  'statecode': 'OR',
  'statepatientnumber': '',
  'statuschangedate': '20/04/2020',
  'typeoftransmission': ''},
 {'agebracket': '55',
  'backupnotes': '',
  'contractedfromwhichpatientsuspected': '',
  'currentstatus': 'Hospitalized',
  'dateannounced': '20/04/2020',
  'detectedcity': '',
  'detecteddistrict': 'Bhadrak',
  'detectedstate': 'Odisha',
  'estimatedonsetdate': '',
  'gender': 'M',
  'nationality': '',
  'notes': 'Details awaited',
  'numcases': '1',
  'patientnumber': '17308',
  'source1': 

In [4]:
#creating a dataframe and transsmitting the raw data into it
full_data = pd.DataFrame.from_dict(data['raw_data'], orient='columns')
#pd.json_normalize(data)
#res = pd.DataFrame.from_dict(data, orient='values')


In [5]:
#setting the index
pd.set_option('display.max_rows', full_data.shape[0]+1)
full_data.head()

,agebracket,backupnotes,contractedfromwhichpatientsuspected,currentstatus,dateannounced,detectedcity,detecteddistrict,detectedstate,estimatedonsetdate,gender,...,notes,numcases,patientnumber,source1,source2,source3,statecode,statepatientnumber,statuschangedate,typeoftransmission
0,48,,,Hospitalized,20/04/2020,,Bhadrak,Odisha,,M,...,Details awaited,1,17307,https://twitter.com/the_hindu/status/125205985...,,,OR,,20/04/2020,
1,55,,,Hospitalized,20/04/2020,,Bhadrak,Odisha,,M,...,Details awaited,1,17308,https://twitter.com/the_hindu/status/125205985...,,,OR,,20/04/2020,
2,57,,,Hospitalized,20/04/2020,,Bhadrak,Odisha,,M,...,Details awaited,1,17309,https://twitter.com/the_hindu/status/125205985...,,,OR,,20/04/2020,
3,58,,,Hospitalized,20/04/2020,,Bhadrak,Odisha,,M,...,Details awaited,1,17310,https://twitter.com/the_hindu/status/125205985...,,,OR,,20/04/2020,
4,66,,,Hospitalized,20/04/2020,,Bhadrak,Odisha,,M,...,Details awaited,1,17311,https://twitter.com/the_hindu/status/125205985...,,,OR,,20/04/2020,


In [6]:
#selecting usefull data(columns) from the main data (covid19india)
full_data2 = full_data.iloc[:,[0,6,13,17,20,1]]

In [7]:
#viewing the whole dataframe
pd.set_option('display.max_rows', full_data2.shape[0]+1)
#res1.head() 
#res1.describe()
#res1.dtypes

In [8]:
#backupnotes is useless rename to spread to, to save list of nodes who got the virus from current node.
#typeoftransmission is useless rename to job, save list of risky jobs at spreading the virus. source: politico.com, bls.gov
full_data2.rename(columns = {'backupnotes':'spreadto'}, inplace = True)
full_data2.rename(columns = {'typeoftransmission':'job'}, inplace = True)
#patientnumber filling nulls
full_data2.patientnumber[10585:10819] = [item for item in range(27892, 28126)]
full_data2.head()

C:\Users\Reza\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\Reza\anaconda3\lib\site-packages\pandas\core\series.py:1042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
C:\Users\Reza\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3341: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  await eval(code_obj, self.user_

,agebracket,detecteddistrict,patientnumber,statecode,job,spreadto
0,48,Bhadrak,17307,OR,,
1,55,Bhadrak,17308,OR,,
2,57,Bhadrak,17309,OR,,
3,58,Bhadrak,17310,OR,,
4,66,Bhadrak,17311,OR,,


In [9]:
#Age filling blanks with Real World probability, leveaning the already filled as it is (covid19india)
# 0–17 years(11%), 18–24 years(14%), 25–44 years(10%), 45–64 years(8%), and ≥65 years(6%). 
# source: https://www.cdc.gov/mmwr/volumes/69/wr/mm6941e1.htm#F1_down
a1=[i for i in range(1,17)]
a2=[i for i in range(18,24)]
a3=[i for i in range(25,44)]
a4=[i for i in range(45,64)]
a5=[i for i in range(65,80)]
age =[a1,a2,a3,a4,a5]
#random weight age choose
full_data2.agebracket[full_data2.agebracket == ''] = full_data2.agebracket.apply(lambda v: random.choices(random.choices(age, weights=(11, 14, 10, 8, 6), k=1)[0], weights=None, k=1)[0])


C:\Users\Reza\anaconda3\lib\site-packages\pandas\core\generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [10]:
#Job column creation filling blanks with Real World probability based on: https://www.bls.gov/oes/2016/may/largest3.htm
job_list = ['Healthcare', 'Resturants&Cashiers', 'Education&Teaching', 'Transportation', 'Building cleanig&staff', 'Other']
full_data2.job = full_data2.job.apply(lambda v: (random.choices(job_list, weights=(6, 9, 6, 7, 4, 33), k=1)[0]))


C:\Users\Reza\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [11]:
#this section is applied after the final processing to see the information of 
#influential nodes given from Final Project Processisng Analysis file

#based on degree centrality results Graphs 1 to 3
#items1 = ['25176','25721', '27837', '20646', '23770', '23824', '18080', '19256', '20604', '20812']
#items2 = ['24841','24839', '24507', '23022', '21458', '26762', '26764', '18970', '18542', '26248']
#items3 = ['24841','24507', '23022', '21458', '23913', '24105', '20671', '17551', '23971', '25567']

#based on eigenvector centrality results Graphs 1 to 3
#items_1 = ['20646','17640', '20588', '21142', '26364', '18919', '17557', '17631', '17571', '20580']
#items_2 = ['18392','18885', '21277', '26248', '18970', '23666', '24626', '24638', '18400', '18933']
#items_3 = ['19663','20739', '22340', '22391', '22481', '24156', '25390', '25723', '25999', '27984']


#Full_items = ['25176','25721', '27837', '20646', '23770', '23824', '18080', '19256', '20604', '20812',
#              '24841','24839', '24507', '23022', '21458', '26762', '26764', '18970', '18542', '26248',
#              '24841','24507', '23022', '21458', '23913', '24105', '20671', '17551', '23971', '25567',
#              '20646','17640', '20588', '21142', '26364', '18919', '17557', '17631', '17571', '20580',
#              '18392','18885', '21277', '26248', '18970', '23666', '24626', '24638', '18400', '18933',
#              '19663','20739', '22340', '22391', '22481', '24156', '25390', '25723', '25999', '27984']
#Analysis_result = full_data2.query("patientnumber in @items_3")
#Analysis_result.to_excel(r'items_eigcent_G.xlsx')
#Analysis_result = full_data2.query("patientnumber in @Full_items")
#Analysis_result.to_excel(r'Full_items.xlsx')
#Analysis_result
#Analysis_result[percent] = (Analysis_result['agebracket'] / Analysis_result['agebracket'].sum()) * 100
#print (100 * Analysis_result['job'].value_counts(normalize=True))
#Analysis_result.ExcelWriter
#Analysis_result.to_excel(r'Analysis_result.xlsx', index = True)

In [12]:
full_data2.rename(columns = {'agebracket':'age_weight'}, inplace = True)
full_data2.rename(columns = {'job':'job_weight'}, inplace = True)


C:\Users\Reza\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [13]:
full_data2["age_weight"]=full_data2["age_weight"].astype('float32')

<ipython-input-13-bdee120a6581>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data2["age_weight"]=full_data2["age_weight"].astype('float32')


In [14]:
# 0–17 years(11%), 18–24 years(14%), 25–44 years(10%), 45–64 years(8%), and ≥65 years(6%). 

for i in range (len(full_data2['age_weight'])):
    if(full_data2['age_weight'][i] >0 and full_data2['age_weight'][i] <= 17):
        full_data2['age_weight'][i]=11
    elif (full_data2['age_weight'][i] >17 and full_data2['age_weight'][i] <= 24):
        full_data2['age_weight'][i]=14
    elif (full_data2['age_weight'][i] >24 and full_data2['age_weight'][i] <= 44):
        full_data2['age_weight'][i]=10
    elif (full_data2['age_weight'][i] >44 and full_data2['age_weight'][i] <= 64):
        full_data2['age_weight'][i]=8
    else :
        full_data2['age_weight'][i]=6

#job_list = ['Healthcare', 'Resturants&Cashiers', 'Education&Teaching', 'Transportation', 'Building cleanig&staff', 'Other']
#res1.job = res1.job.apply(lambda v: (random.choices(job_list, weights=(6, 9, 6, 7, 4, 33), k=1)[0]))

for i in range (len(full_data2['job_weight'])):
    if(full_data2['job_weight'][i] == 'Healthcare'):
        full_data2['job_weight'][i]=6
    elif (full_data2['job_weight'][i] == 'Resturants&Cashiers'):
        full_data2['job_weight'][i]=9
    elif (full_data2['job_weight'][i] == 'Education&Teaching'):
        full_data2['job_weight'][i]=6
    elif (full_data2['job_weight'][i] == 'Transportation'):
        full_data2['job_weight'][i]=7
    elif (full_data2['job_weight'][i] == 'Building cleanig&staff'):
        full_data2['job_weight'][i]=4
    else :
        full_data2['job_weight'][i]=4


<ipython-input-14-d6aab64bcf01>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data2['age_weight'][i]=8
C:\Users\Reza\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-14-d6aab64bcf01>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data2['age_weight'][i]=6
<ipython-input-14-d6aab64bcf01>:9: Set

In [15]:
full_data2["job_weight"]=full_data2["job_weight"].astype('float32')

<ipython-input-15-c93e056ad889>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data2["job_weight"]=full_data2["job_weight"].astype('float32')


In [16]:


#res3["patientnumber"]=res3["patientnumber"].apply(lambda x: int(float(x)))
full_data2["detecteddistrict"] = full_data2["detecteddistrict"].astype('str') 
full_data2['agg_risk'] = full_data2['age_weight']+ full_data2['job_weight']


<ipython-input-16-d8851f0bd561>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data2["detecteddistrict"] = full_data2["detecteddistrict"].astype('str')
<ipython-input-16-d8851f0bd561>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data2['agg_risk'] = full_data2['age_weight']+ full_data2['job_weight']


In [17]:
full_data2.dtypes

age_weight          float32
detecteddistrict     object
patientnumber        object
statecode            object
job_weight          float32
spreadto             object
agg_risk            float32
dtype: object

In [18]:
data = []

In [19]:
for i in range(full_data2['spreadto'].shape[0]):
        data.append([])

In [20]:
full_data2['spreadto'] =data

<ipython-input-20-9423f9f542ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data2['spreadto'] =data


In [21]:
tt=full_data2

In [22]:
tt['patient_index']=tt['patientnumber']

<ipython-input-22-70e96a6d83fb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tt['patient_index']=tt['patientnumber']


In [23]:
full_data2.head()

,age_weight,detecteddistrict,patientnumber,statecode,job_weight,spreadto,agg_risk,patient_index
0,8.0,Bhadrak,17307,OR,6.0,[],14.0,17307
1,8.0,Bhadrak,17308,OR,6.0,[],14.0,17308
2,8.0,Bhadrak,17309,OR,4.0,[],12.0,17309
3,8.0,Bhadrak,17310,OR,6.0,[],14.0,17310
4,6.0,Bhadrak,17311,OR,4.0,[],10.0,17311


In [24]:
tt=tt.set_index('patient_index')

In [25]:
tt.head()

,age_weight,detecteddistrict,patientnumber,statecode,job_weight,spreadto,agg_risk
patient_index,,,,,,,
17307,8.0,Bhadrak,17307,OR,6.0,[],14.0
17308,8.0,Bhadrak,17308,OR,6.0,[],14.0
17309,8.0,Bhadrak,17309,OR,4.0,[],12.0
17310,8.0,Bhadrak,17310,OR,6.0,[],14.0
17311,6.0,Bhadrak,17311,OR,4.0,[],10.0


In [26]:
groups=tt.groupby(['statecode','detecteddistrict'])

In [27]:
print(groups.count().count())

age_weight       283
patientnumber    283
job_weight       283
spreadto         283
agg_risk         283
dtype: int64


In [28]:
full_data_indexed=full_data2.set_index(['statecode','detecteddistrict'])

In [29]:
unique_index = set(full_data_indexed.index)

In [30]:
len(unique_index)

283

In [31]:
for ind in unique_index:
    curr_group = groups.get_group(ind)
    if curr_group.shape[0]>1:
        risk_thresh = curr_group['agg_risk'].median()
        risk_group = curr_group[curr_group['agg_risk']>=risk_thresh]
        #curr_group.set_index('patientnumber', inplace=True)
        #risk_group.set_index('patientnumber', inplace=True)
        sample_risk=risk_group.sample(math.ceil(10*risk_group.shape[0]/100))
        #main_sample=curr_group.sample(math.ceil(10*curr_group.shape[0]/100))
        max_pop=math.ceil(20*curr_group.shape[0]/100)
        #spreader = curr_group[curr_group.index.isin(sample_risk.index)]
        risk_sum = sample_risk['agg_risk'].sum()
        for patient in sample_risk.index:
            spread_portion = sample_risk.loc[patient].agg_risk/risk_sum

            victim_portion = spread_portion*max_pop
            victims=curr_group.sample(math.ceil(victim_portion))
        
            result = victims["patientnumber"].values
            res_str = ""
            for r in result:
                if r!= patient:
                  curr_group.loc[patient, "spreadto"].append(r)
 
    

In [34]:
tt.head(10)

,age_weight,detecteddistrict,patientnumber,statecode,job_weight,spreadto,agg_risk
patient_index,,,,,,,
17307,8.0,Bhadrak,17307,OR,6.0,[],14.0
17308,8.0,Bhadrak,17308,OR,6.0,[],14.0
17309,8.0,Bhadrak,17309,OR,4.0,[],12.0
17310,8.0,Bhadrak,17310,OR,6.0,[],14.0
17311,6.0,Bhadrak,17311,OR,4.0,[],10.0
17312,10.0,Balasore,17312,OR,4.0,[],14.0
17313,11.0,Balasore,17313,OR,4.0,[17312],15.0
17314,10.0,,17314,WB,9.0,[],19.0
17315,11.0,,17315,WB,9.0,"[17338, 21377, 21379, 24486, 20089]",20.0


In [35]:
risk1 = tt['agg_risk'].median()
risk_group1 = tt[tt['agg_risk']>=risk1]

sample_risk1=risk_group1.sample(math.ceil(10*risk_group1.shape[0]/100))
max_infected_num = math.ceil(20*tt.shape[0]/100)
risk_sum = sample_risk1['agg_risk'].sum()

for patient_index in sample_risk1.index:
    spread_portion = sample_risk1.loc[patient_index].agg_risk/risk_sum

    victim_portion = spread_portion*max_infected_num
    victims=tt.sample(math.ceil(victim_portion))

    result = victims["patientnumber"].values
    for r in result:
        #if r!= patient_index:
        tt.loc[patient_index, "spreadto"].append(r)

In [36]:
tt.head(10)

,age_weight,detecteddistrict,patientnumber,statecode,job_weight,spreadto,agg_risk
patient_index,,,,,,,
17307,8.0,Bhadrak,17307,OR,6.0,[],14.0
17308,8.0,Bhadrak,17308,OR,6.0,[],14.0
17309,8.0,Bhadrak,17309,OR,4.0,[],12.0
17310,8.0,Bhadrak,17310,OR,6.0,[],14.0
17311,6.0,Bhadrak,17311,OR,4.0,[],10.0
17312,10.0,Balasore,17312,OR,4.0,[],14.0
17313,11.0,Balasore,17313,OR,4.0,[17312],15.0
17314,10.0,,17314,WB,9.0,[],19.0
17315,11.0,,17315,WB,9.0,"[17338, 21377, 21379, 24486, 20089]",20.0


In [37]:
my_network = tt[['patientnumber','spreadto']]

In [38]:
print(my_network.shape)

(10819, 2)


In [39]:
G_data = []
for i in my_network.index:
  infected = my_network.loc[i,'spreadto']
  if len(infected)>0:  
    for j in infected:
      temp=(i,j)
      G_data.append(temp)
  else:
      temp =(i,i)
      G_data.append(temp)

In [40]:
len(G_data)

14423

In [42]:
#res5= res4.iloc[:,[2,5]]
#res5.head(1000)

#print(path+"file.txt")

In [44]:
with open("file.txt", 'w') as output:
    for row in G_data:
        output.write(str(row) + '\n')